# DVOACAP-Python: Basic Prediction Example

This notebook demonstrates how to use DVOACAP-Python to predict HF radio propagation between two locations.

## What You'll Learn
- How to set up a basic prediction
- Understanding prediction results
- Interpreting SNR, reliability, and MUF
- Comparing different frequencies

## Setup

First, import the required modules:

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

import numpy as np
import matplotlib.pyplot as plt
from src.dvoacap.path_geometry import GeoPoint
from src.dvoacap.prediction_engine import PredictionEngine

print("DVOACAP-Python loaded successfully!")

## Example 1: Single Frequency Prediction

Let's predict propagation from **Philadelphia, PA** to **London, UK** on **14 MHz** at **12:00 UTC**.

In [ ]:
# Create prediction engine
engine = PredictionEngine()

# Set transmitter location (Philadelphia)
tx = GeoPoint.from_degrees(39.9526, -75.1652)
engine.params.tx_location = tx
engine.params.tx_power = 100  # 100 watts

# Set receiver location (London)
rx = GeoPoint.from_degrees(51.5074, -0.1278)

# Set propagation conditions
engine.params.month = 6  # June
engine.params.ssn = 100  # Sunspot number (moderate solar activity)

# Run prediction for 14 MHz at 12:00 UTC
engine.predict(rx_location=rx, utc_time=12.0/24.0, frequencies=[14.0])

# Display results
if len(engine.predictions) > 0:
    pred = engine.predictions[0]
    print("=" * 60)
    print("PREDICTION RESULTS")
    print("=" * 60)
    print(f"Path: Philadelphia, PA → London, UK")
    print(f"Distance: {engine.path.dist:.0f} km")
    print(f"Frequency: 14.0 MHz")
    print(f"Time: 12:00 UTC")
    print(f"Month: June")
    print(f"SSN: 100")
    print()
    print(f"Mode: {pred.get_mode_name(engine.path.dist)}")
    print(f"SNR: {pred.signal.snr_db:.1f} dB")
    print(f"Reliability: {pred.signal.reliability * 100:.1f}%")
    print(f"Total Loss: {pred.signal.total_loss_db:.1f} dB")
    print(f"MUF Factor: {pred.signal.muf_day:.2f}")
    print(f"TX Angle: {np.rad2deg(pred.tx_elevation):.1f}°")
    print(f"Virtual Height: {pred.virt_height:.1f} km")
    print("=" * 60)
else:
    print("No valid propagation path found")

## Example 2: Frequency Scan

Let's compare multiple frequencies to find the best one for this path:

In [ ]:
# HF band frequencies (MHz)
frequencies = [3.5, 7.0, 10.1, 14.0, 18.1, 21.0, 24.9, 28.0]

# Storage for results
results = []

# Run predictions for each frequency
for freq in frequencies:
    engine.predict(rx_location=rx, utc_time=12.0/24.0, frequencies=[freq])
    
    if len(engine.predictions) > 0:
        pred = engine.predictions[0]
        results.append({
            'freq': freq,
            'snr': pred.signal.snr_db,
            'reliability': pred.signal.reliability * 100,
            'mode': pred.get_mode_name(engine.path.dist)
        })

# Display results table
print("\nFREQUENCY COMPARISON")
print("=" * 60)
print(f"{'Frequency':<12} {'SNR (dB)':<12} {'Reliability':<15} {'Mode'}")
print("=" * 60)
for r in results:
    print(f"{r['freq']:<12.1f} {r['snr']:<12.1f} {r['reliability']:<15.1f} {r['mode']}")
print("=" * 60)

## Example 3: Visualizing Results

Let's create a plot showing SNR vs Frequency:

In [ ]:
# Extract data for plotting
freqs = [r['freq'] for r in results]
snrs = [r['snr'] for r in results]
rels = [r['reliability'] for r in results]

# Create plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# SNR plot
ax1.plot(freqs, snrs, 'bo-', linewidth=2, markersize=8)
ax1.grid(True, alpha=0.3)
ax1.set_xlabel('Frequency (MHz)', fontsize=12)
ax1.set_ylabel('SNR (dB)', fontsize=12)
ax1.set_title('Signal-to-Noise Ratio vs Frequency\nPhiladelphia → London (12:00 UTC, June, SSN=100)', fontsize=14)
ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='0 dB threshold')
ax1.legend()

# Reliability plot
ax2.plot(freqs, rels, 'go-', linewidth=2, markersize=8)
ax2.grid(True, alpha=0.3)
ax2.set_xlabel('Frequency (MHz)', fontsize=12)
ax2.set_ylabel('Reliability (%)', fontsize=12)
ax2.set_title('Reliability vs Frequency', fontsize=14)
ax2.axhline(y=50, color='r', linestyle='--', alpha=0.5, label='50% threshold')
ax2.set_ylim(0, 100)
ax2.legend()

plt.tight_layout()
plt.show()

## Example 4: 24-Hour Time Scan

Let's see how propagation changes throughout the day on 14 MHz:

In [ ]:
# Run predictions for each hour
hours = list(range(24))
time_results = []

for hour in hours:
    engine.predict(rx_location=rx, utc_time=hour/24.0, frequencies=[14.0])
    
    if len(engine.predictions) > 0:
        pred = engine.predictions[0]
        time_results.append({
            'hour': hour,
            'snr': pred.signal.snr_db,
            'reliability': pred.signal.reliability * 100
        })

# Plot 24-hour variation
hours_list = [r['hour'] for r in time_results]
snrs_list = [r['snr'] for r in time_results]
rels_list = [r['reliability'] for r in time_results]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# SNR over 24 hours
ax1.plot(hours_list, snrs_list, 'b-', linewidth=2)
ax1.fill_between(hours_list, snrs_list, alpha=0.3)
ax1.grid(True, alpha=0.3)
ax1.set_xlabel('UTC Hour', fontsize=12)
ax1.set_ylabel('SNR (dB)', fontsize=12)
ax1.set_title('24-Hour SNR Variation (14 MHz)\nPhiladelphia → London (June, SSN=100)', fontsize=14)
ax1.set_xticks(range(0, 25, 2))
ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5)

# Reliability over 24 hours
ax2.plot(hours_list, rels_list, 'g-', linewidth=2)
ax2.fill_between(hours_list, rels_list, alpha=0.3)
ax2.grid(True, alpha=0.3)
ax2.set_xlabel('UTC Hour', fontsize=12)
ax2.set_ylabel('Reliability (%)', fontsize=12)
ax2.set_title('24-Hour Reliability Variation', fontsize=14)
ax2.set_xticks(range(0, 25, 2))
ax2.set_ylim(0, 100)
ax2.axhline(y=50, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Understanding the Results

### SNR (Signal-to-Noise Ratio)
- **> 10 dB**: Excellent signal
- **0-10 dB**: Good signal
- **< 0 dB**: Weak/marginal signal

### Reliability
- **> 90%**: Very reliable
- **50-90%**: Moderately reliable
- **< 50%**: Unreliable

### MUF Factor
- **0.5-0.85**: Good operating range
- **> 0.85**: Near MUF (frequency may be too high)
- **< 0.5**: Well below MUF (may have higher absorption)

### Propagation Modes
- **E**: E-layer single hop
- **F1**: F1-layer propagation
- **F2**: F2-layer propagation (most common for long paths)
- **2F2**: Two-hop F2 propagation
- **3F2**: Three-hop F2 propagation

## Next Steps

Try these exercises:
1. Change the transmit power and observe the effect on SNR
2. Try different months to see seasonal variation
3. Experiment with different SSN values (10=solar min, 200=solar max)
4. Try different geographic paths (short, medium, long)
5. Explore the `02_parameter_sensitivity.ipynb` notebook for more advanced analysis